<a href="https://colab.research.google.com/github/TiwariKishan/Algorithms/blob/master/Word2vec_Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string

In [0]:
data = pd.read_csv('/content/drive/My Drive/english_dataset.tsv', header = 0, sep='\t')
data=data.dropna()

In [57]:
data.columns = ['text_id','text', 'task_1','task_2','task_3']
data=data.drop(['text_id'],axis=1)
data=data.drop(['task_2'],axis=1)
data=data.drop(['task_3'],axis=1)
data.task_1.unique()
data.head(500)

,text,task_1
0,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT
1,@politico No. We should remember very clearly ...,HOF
2,@cricketworldcup Guess who would be the winner...,NOT
3,Corbyn is too politically intellectual for #Bo...,NOT
4,All the best to #TeamIndia for another swimmin...,NOT
...,...,...
495,"#fucktrump #impeachtrump 😂😂😂😂😂😂😂 @ Houston, Te...",NOT
496,“Every time I see Boris he’s hanging from a wi...,NOT
497,This one is understandable. #dickhead! https:/...,NOT
498,#trendingnow #terrorist #murderer #dumptrump ...,NOT


In [0]:
data['task_1'] = data['task_1'].map({'NOT': 0, 'HOF': 1})

In [59]:
data.task_1.unique()

array([0, 1])

In [60]:
data.shape

(5852, 2)

In [0]:
pos = []
neg = []
for l in data.task_1:
    if l == 0:
        pos.append(0)
        neg.append(1)
    elif l == 1:
        pos.append(1)
        neg.append(0)

In [0]:
data['Pos']= pos
data['Neg']= neg

In [63]:
data.head()

,text,task_1,Pos,Neg
0,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,0,0,1
1,@politico No. We should remember very clearly ...,1,1,0
2,@cricketworldcup Guess who would be the winner...,0,0,1
3,Corbyn is too politically intellectual for #Bo...,0,0,1
4,All the best to #TeamIndia for another swimmin...,0,0,1


In [0]:
def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct

data['Text_Clean'] = data['text'].apply(lambda x: remove_punct(x))

In [65]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
from nltk import word_tokenize, WordNetLemmatizer
tokens = [word_tokenize(sen) for sen in data.Text_Clean]

In [0]:
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens]

In [68]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

Function for removing all stopwords from the tokens


In [0]:
def remove_stop_words(tokens): 
    return [word for word in tokens if word not in stoplist]

In [0]:
filtered_words = [remove_stop_words(sen) for sen in lower_tokens]

join the words

In [0]:
result = [' '.join(sen) for sen in filtered_words]

In [0]:
data['Text_Final'] = result

In [0]:
data['tokens'] = filtered_words

In [0]:
data = data[['Text_Final', 'tokens', 'task_1', 'Pos', 'Neg']]

In [76]:
data[:47]

,Text_Final,tokens,task_1,Pos,Neg
0,dhonikeepstheglove watch sports minister kiren...,"[dhonikeepstheglove, watch, sports, minister, ...",0,0,1
1,politico remember clearly individual1 admitted...,"[politico, remember, clearly, individual1, adm...",1,1,0
2,cricketworldcup guess would winner cwc19 team ...,"[cricketworldcup, guess, would, winner, cwc19,...",0,0,1
3,corbyn politically intellectual borisjohnsonsh...,"[corbyn, politically, intellectual, borisjohns...",0,0,1
4,best teamindia another swimming competition su...,"[best, teamindia, another, swimming, competiti...",0,0,1
5,kellymiller513 therealoj32 hope remembered wip...,"[kellymiller513, therealoj32, hope, remembered...",0,0,1
6,icc latest design wc2019 trophy cwc2019 cwc19 ...,"[icc, latest, design, wc2019, trophy, cwc2019,...",0,0,1
7,ados trendingnow blacklivesmatter justice fuck...,"[ados, trendingnow, blacklivesmatter, justice,...",1,1,0
8,thanks support wow 600k graffiti massive impac...,"[thanks, support, wow, 600k, graffiti, massive...",0,0,1
9,wearing balidaanbadge gloves msdhoni shown lov...,"[wearing, balidaanbadge, gloves, msdhoni, show...",0,0,1


Split the data into trainning and testing

In [0]:
data_train, data_test = train_test_split(data, test_size=0.10, random_state=54)

In [78]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

81959 words total, with a vocabulary size of 19925
Max sentence length is 77


In [79]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

9041 words total, with a vocabulary size of 4277
Max sentence length is 68


In [0]:
word2vec_path = '/content/drive/My Drive/General/GoogleNews-vectors-negative300.bin.gz'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

Embedding

In [0]:
training_embeddings = get_word2vec_embeddings(word2vec, data_train, generate_missing=True)

In [0]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300

In [0]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["Text_Final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

In [0]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

In [0]:
test_sequences = tokenizer.texts_to_sequences(data_test["Text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
   def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index): 
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)

    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [0]:
label_names = ['Pos', 'Neg']

In [0]:
y_train = data_train[label_names].values

In [0]:
x_train = train_cnn_data
y_tr = y_train

In [0]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))

In [0]:
num_epochs = 6
batch_size = 23

In [0]:
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

In [0]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)

In [0]:
labels = [1, 0]

In [0]:
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])

In [0]:
sum(data_test.task_1==prediction_labels)/len(prediction_labels)

In [0]:
data_test.task_1.value_counts()

In [0]:
from sklearn.metrics import classification_report,confusion_matrix

In [0]:
print(classification_report(data_test.task_1,prediction_labels))